# Task 2


In [ ]:
import os
from functools import partial

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import layers

from utils import get_dirs, train_test_split

Load the dataset

In [ ]:
DIRS = get_dirs(os.path.abspath('') + os.sep + 'Task2.ipynb')

In [ ]:
images = np.load(DIRS['data'] + 'images.npy')
labels = np.load(DIRS['data'] + 'labels.npy')

# show 9 random images with plt.imshow
fig, axes = plt.subplots(3, 3, figsize=(5, 5))
for i, ax in enumerate(axes.flat):
    ri = np.random.randint(0, images.shape[0])
    ax.imshow(images[ri], cmap='gray')
    ax.set(xticks=[], yticks=[])
    ax.set_title(f'{labels[ri][0]}:{labels[ri][1]}')

We can see that the dataset consists of images of clocks, each with a different angle, rotation and lightning. This makes it rather difficult to train a model to classify the images.

We will be trying multiple ways of labelling the clocks, firstly try by regression label.

In [ ]:
regressionLabels = labels[:, 0] + labels[:, 1] / 60

X_train, X_test, y_train, y_test = train_test_split(X=images, y=regressionLabels, test_size=0.2)

X_train.shape

In [ ]:
def lossFunction(y_true, y_pred):
    return tf.math.minimum(tf.math.abs(y_true - y_pred), 12 - tf.math.abs(y_true - y_pred))

In [ ]:
def activationFunction(x):
    return keras.activations.relu(x, alpha=0.0, max_value=12)

In [ ]:
a = tf.constant([-3, -1, 0, 1 , 3, 100])
b = activationFunction(a)
tf.print(b)

In [ ]:
# base architecture (from Luuk)
DefaultConv2D = partial(layers.Conv2D, kernel_initializer='he_normal', kernel_size = 3, activation = 'elu', padding = 'SAME')
model = keras.Sequential([
    DefaultConv2D(filters = 32, kernel_size = 5, strides = (3, 3), input_shape = [150, 150, 1]),
    layers.MaxPooling2D(pool_size = 2),
    DefaultConv2D(filters = 64),
    layers.MaxPooling2D(pool_size = 2),
    DefaultConv2D(filters = 128),
    layers.MaxPooling2D(pool_size = 2),
    layers.Flatten()
])

In [ ]:
model.add(layers.Dense(1, activation = activationFunction))

In [ ]:
model.compile(loss = lossFunction, optimizer = 'adam', metrics = ['mae'])

In [ ]:
model.fit(X_train, y_train, epochs = 5, validation_data = (X_test, y_test), verbose=1)

In [ ]:
model.compile(loss=lossFunction, optimizer='adam', metrics=['mean_squared_error'])

history = model.fit(X_train, y_train, batch_size=32, epochs=3, validation_split=0.1, verbose=1)

#model.evaluate(X_test, y_test, verbose=2)

#show differences between predicted and actual values
# add straight line to show perfect prediction

y_pred = model.predict(X_test)
plt.figure(figsize=(10,10))
plt.scatter(y_test, y_pred)
plt.plot([0, 24], [0, 24], color='red')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.show()